This notebook is used to test recommandation based on links present on a pages. 

In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
%config InlineBackend.figure_formats=['svg']

from pymongo import MongoClient

from IPython.display import display, HTML

from wekeypedia.wikipedia.page import WikipediaPage as Page

import pandas as pd
import datetime as dt

#import json
#import os
#import hashlib
#import sys
#import math
#import codecs
#import numpy as np
#import seaborn as sns
#import datetime
#import matplotlib as mpl
#import matplotlib.pyplot as plt
#import matplotlib.dates as dates
#import networkx as nx

In [2]:
client = MongoClient('localhost', 27017)
dbwekey = client.get_database('wekeypedia')

clickstream_col = dbwekey.get_collection('clickstream')


info_col = dbwekey.get_collection('info')
links_col = dbwekey.get_collection('links')
backlinks_col = dbwekey.get_collection('backlinks')
pageviews_col = dbwekey.get_collection('pageviews')
caegories_col = dbwekey.get_collection('categories')
revisions_col = dbwekey.get_collection('revisions')


In [3]:
dbwekey.collection_names()

[u'system.indexes',
 u'clickstream',
 u'info',
 u'pageviews',
 u'links',
 u'backlinks',
 u'recommendation',
 u'revisions']

In [4]:
#print {item['title']:item['click'] for item in clickstream_col.find_one({'title':'Algorithm'}).get('links')}

The next function are used to download data. The jar used are generated used the java toolkit.

Can be improve by imporving the manipulation of the java toolkit by command line.

In [5]:
from subprocess import Popen
from subprocess import PIPE

def dl_pages_data(jar_file,collection,list_of_page_names):
    proc = Popen(["java","-jar",jar_file,"-mongo","-nbThread","7"],stdin=PIPE)#,stdout=open('debug.txt','wb'))
    proc.stdin.write('\n'.join(list_of_page_names).encode('utf8'))
    proc.stdin.close()
    proc.wait()

def dl_pages_info(list_of_page_names):
    dl_pages_data("jar/info.jar",info_col,list_of_page_names)

def dl_pages_links(list_of_page_names):
    dl_pages_data("jar/links.jar",links_col,list_of_page_names)

def dl_pages_backlinks(list_of_page_names):
    dl_pages_data("jar/backlinks.jar",backlinks_col,list_of_page_names)

def dl_pages_pageviews(list_of_page_names):
    dl_pages_data("jar/pageviews.jar",pageviews_col,list_of_page_names)

def dl_pages_revisions(list_of_page_names):
    dl_pages_data("jar/revisions.jar",revisions_col,list_of_page_names)

def dl_pages_categories(list_of_page_names):
    dl_pages_data("jar/categories.jar",categories_col,list_of_page_names)

def dl_resolve_redirects(list_of_pages):
    dl_pages_info(list_of_pages)
    res=set()
    for page_name,page in get_pages_info(list_of_pages).items():
        if page!=None and 'missing' not in page:
            if u'redirects_to' in page:
                res.add(page[u'redirects_to'])
            else:
                res.add(page[u'title'])
    return(res)
        

    
    
def get_pages_data(collection,list_of_page_names):
    res={}
    for page in list_of_page_names:
        res[page]=collection.find_one({'title':page})
    return res

def get_pages_info(list_of_page_names):
    return get_pages_data(info_col,list_of_page_names)

def get_pages_links(list_of_page_names):
    return get_pages_data(links_col,list_of_page_names)

def get_pages_backlinks(list_of_page_names):
    return get_pages_data(backlinks_col,list_of_page_names)

def get_pages_pageviews(list_of_page_names):
    return get_pages_data(pageviews_col,list_of_page_names)
 
def get_pages_revisions(list_of_page_names):
    return get_pages_data(revisions_col,list_of_page_names)

def get_pages_categories(list_of_page_names):
    return get_pages_data(categories_col,list_of_page_names)
    
def resolve_redirects(list_of_pages):
    res=set()
    for page_name,page in get_pages_info(list_of_pages).items():
        if page!=None and 'missing' not in page:
            if u'redirects_to' in page:
                res.add(page[u'redirects_to'])
            else:
                res.add(page[u'title'])
    return(res)




#dest='Algorithm'
#print [ x['title'] for x in get_pages_backlinks([dest])[dest]['backlinks'] ]
#dic = get_pages_pageviews([dest])[dest]['pageviews']
#print sorted(dic.items(), key=lambda x:x[1], reverse=True)
#print sum([v for k,v in get_pages_pageviews([dest])[dest]['pageviews'].items()])

In [ ]:
To download the graph local to a page. 

In [6]:
def dl_local_graph(page_name):
    redir=dl_resolve_redirects([page_name])
    if len(redir)==0:
        return
    dest=list(redir)[0]  
    dl_pages_links([dest])   
    dl_pages_backlinks([dest])

    dl_resolve_redirects(set( [ x['title'] for x in get_pages_links([dest])[dest]['links'] ] ))
    backlinks = dl_resolve_redirects(set ( [ x['title'] for x in get_pages_backlinks([dest])[dest]['backlinks'] ] ))

    print "nb backlinks",len(backlinks),
    dl_pages_pageviews(backlinks)

    dl_pages_links(backlinks)
    redirects=set()
    for page in backlinks:
        redirects = redirects.union([ x['title'] for x in get_pages_links([page])[page]['links'] ])
    print "nb redirects: ",len(redirects),
    dl_resolve_redirects(redirects)


def get_ranking(page_name):
    redir=dl_resolve_redirects([page_name])
    if len(redir)==0:
        return []
    dest=list(redir)[0]      
    links = resolve_redirects(set ( [ x['title'] for x in get_pages_links([dest])[dest]['links'] ] ))
    if dest in links: links.remove(page_name)
    res = {}
    for li in links:
        res[li]=0
        
    backlinks = resolve_redirects(set ( [ x['title'] for x in get_pages_backlinks([dest])[dest]['backlinks'] ] ))
    backlinks.remove(page_name)
    for ba in backlinks:
        links_ba = resolve_redirects(set ( [ x['title'] for x in get_pages_links([ba])[ba]['links'] ] ))
        pageviews_ba = sum([v for k,v in get_pages_pageviews([ba])[ba]['pageviews'].items()])
        for li in links & links_ba:
            res[li]+=pageviews_ba

    return(sorted(res.items(), key=lambda x:x[1], reverse=True))
        
    
#dl_local_graph('Algorithm')

#print 'finished dl...' 
#res = get_ranking('Algorithm')
#print get_pages_info(['Properties of algorithms'])

In [13]:
#get_pages_links(['Travian'])['Travian']

The next function allow to get a ranking of the pages by the pageviews.

In [ ]:
def get_pageviews_ranking(page_name):
    redir=dl_resolve_redirects([page_name])
    if len(redir)==0:
        return []
    dest=list(redir)[0] 
    dl_pages_revisions(dest)
    contributors = set( [x['user'] for x in get_pages_revisions([dest])[dest]['revisions'] 
                        if x['userid']!=0 and 'BOT' not in x['user'].upper()] ) 
    links = resolve_redirects(set ( [ x['title'] for x in get_pages_links([dest])[dest]['links'] ] ))
    dl_pages_revisions(links)
    revisions = get_pages_revisions(links)
    
    res = {}
    for page in revisions:
        try:
            contrib = set( [x['user'] for x in revisions[page]['revisions'] 
                            if 'user' in x and x['userid']!=0 and 'BOT' not in x['user'].upper()] )
            res[page]=len(contributors & contrib)/(len(contributors | contrib)+0.0)
        except:
            print page
            
    return(sorted(res.items(), key=lambda x:x[1], reverse=True))

In [11]:
def get_revisions_ranking(page_name):
    redir=dl_resolve_redirects([page_name])
    if len(redir)==0:
        return []
    dest=list(redir)[0] 
    dl_pages_revisions(dest)
    contributors = set( [x['user'] for x in get_pages_revisions([dest])[dest]['revisions'] 
                        if x['userid']!=0 and 'BOT' not in x['user'].upper()] ) 
    links = resolve_redirects(set ( [ x['title'] for x in get_pages_links([dest])[dest]['links'] ] ))
    dl_pages_revisions(links)
    revisions = get_pages_revisions(links)
    
    res = {}
    for page in revisions:
        try:
            contrib = set( [x['user'] for x in revisions[page]['revisions'] 
                            if 'user' in x and x['userid']!=0 and 'BOT' not in x['user'].upper()] )
            res[page]=len(contributors & contrib)#/(len(contributors | contrib)+0.0)
        except:
            print page
            
    return(sorted(res.items(), key=lambda x:x[1], reverse=True))

#tmp = get_revisions_ranking('Algorithm')

#basic functions

In [15]:
def get_clicks_ranking(page_name):
    global clickstream_col
    page = clickstream_col.find_one({"title":page_name})
    if page!=None:
        res={}
        for cl in page.get('links'):
            dest = resolve_redirects( [cl['title'].replace('_',' ')] )
            if len(dest)>0:
                dest = list(dest )[0]
                res[dest]=int(cl.get('click'))
        return sorted(res.items(), key=lambda x:x[1], reverse=True)
    return page

#get_clicks('Travian')

In [8]:
#get_ranking('Travian')

In [28]:
#print get_clicks( 'Algorithm')

#Testing recommandation

In [ ]:
dl_pages_pageviews([k for (k,v) in dic_1])
tmp = get_pages_pageviews([k for (k,v) in dic_1])
dic_2={}
for key in tmp:
    try:
        dic_2[key]=sum([v for k,v in tmp[key]["pageviews"].items()])
    except:
        print k
print(len(dic_2))

dic_3={k:v for (k,v) in get_revisions_ranking('Algorithm')}
print len(dic_3)

In [16]:
dic_1=get_clicks_ranking('Pi')
print (len(dic_1))
dic_2={}
dic_3={}


html = "<table>"
html+= "<tr><td>Title</td><td>Dic 1</td><td>Dic 2</td><td>Dic 3</td></tr>"
for (k,v) in dic_1[:30]:
    html += "<tr><td><a href=\"https://en.wikipedia.org/wiki/%s\">%s</a></td><td>%s</td>" % (k,k,v)
    if k in dic_2:
        html += "<td>%s</td>" % (dic_2[k])   
    else:
        html += "<td> -1 </td>"   
    if k in dic_3:
        html += "<td>%s</td>" % (dic_3[k])   
    else:
        html += "<td> <3 </td>"   
    html += "</tr>"
html += "</table>"    

display(HTML(html))

221


Title,Dic 1,Dic 2,Dic 3
Pi (disambiguation),1739,-1,<3
Squaring the circle,1513,-1,<3
Irrational number,1198,-1,<3
Pi (letter),1046,-1,<3
Proof that π is irrational,786,-1,<3
Transcendental number,765,-1,<3
Pi Day,729,-1,<3
Chronology of computation of π,727,-1,<3
Circumference,700,-1,<3
Approximations of π,675,-1,<3
